In [1]:
dfraces = spark.sql("SELECT * FROM MLModel.races")

StatementMeta(, 02e1acd0-d278-463d-8602-26e0bb581396, 3, Finished, Available, Finished)

In [2]:
dfresults = spark.sql("SELECT * FROM MLModel.results")

StatementMeta(, 02e1acd0-d278-463d-8602-26e0bb581396, 4, Finished, Available, Finished)

#daniel  
añadí el filtro del año  
ND ra.year >= 1982

In [ ]:
dfracesresults = spark.sql("""
    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,
    re.grid,cast(re.position as int) as position,re.points,re.statusId,st.status,
    re.laps,re.fastestLap,re.fastestLapTime
    FROM MLModel.races ra 
    JOIN MLModel.results re ON ra.raceId = re.raceId
    JOIN MLModel.status st ON st.statusId= re.statusId
    WHERE position in (1,2,3) AND ra.year >= 1982 
    ORDER BY ra.year desc,ra.round, position
""")

StatementMeta(, 119f8409-6e7a-4a17-8300-37416da1d7a4, 6, Finished, Available, Finished)

In [1]:
def generate_dataset(yearfrom,yearuntil,positionfrom,positionuntil):
    df = spark.sql( f"""
    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,
    re.grid,cast(re.position as int) as position,re.points,re.statusId,st.status,
    re.laps,re.fastestLap,re.fastestLapTime
    FROM races ra 
    JOIN results re ON ra.raceId = re.raceId
    JOIN status st ON st.statusId= re.statusId
    WHERE position >= {positionfrom} AND position <= {positionuntil} AND ra.year >= {yearfrom} AND ra.year <= {yearuntil} 
    ORDER BY ra.year desc,ra.round, position""")
    return df

StatementMeta(, 804234f2-b360-4def-8109-0034b1659844, 3, Finished, Available, Finished)

In [ ]:
dfracesresults.show(10)

StatementMeta(, 119f8409-6e7a-4a17-8300-37416da1d7a4, 8, Finished, Available, Finished)

+------+----+-----+---------+--------+-------------+----+--------+------+--------+--------+----+----------+--------------+
|raceId|year|round|circuitId|driverId|constructorId|grid|position|points|statusId|  status|laps|fastestLap|fastestLapTime|
+------+----+-----+---------+--------+-------------+----+--------+------+--------+--------+----+----------+--------------+
|  1121|2024|    1|        3|     830|            9|   1|       1|  26.0|       1|Finished|  57|        39|      1:32.608|
|  1121|2024|    1|        3|     815|            9|   5|       2|  18.0|       1|Finished|  57|        40|      1:34.364|
|  1121|2024|    1|        3|     832|            6|   4|       3|  15.0|       1|Finished|  57|        44|      1:34.507|
|  1122|2024|    2|       77|     830|            9|   1|       1|  25.0|       1|Finished|  50|        50|      1:31.773|
|  1122|2024|    2|       77|     815|            9|   3|       2|  18.0|       1|Finished|  50|        37|      1:32.273|
|  1122|2024|   

In [24]:
dfracesresults.write.format("delta").saveAsTable("racesresults")

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
